In [1]:
import pandas as pd
import numpy as np

# Import applications dataset
application_record = pd.read_csv("data/application_record.csv")
application_record.shape

(438557, 18)

In [2]:
# Remove Duplicates
application_record.drop_duplicates(subset ="ID", keep = False, inplace = True)

In [3]:
# Remove Y with 1 and N with 0
application_record['FLAG_OWN_CAR'] = np.where(application_record['FLAG_OWN_CAR'].eq('Y', 'N'), 1, 0)
application_record['FLAG_OWN_REALTY'] = np.where(application_record['FLAG_OWN_REALTY'].eq('Y', 'N'), 1, 0)

In [4]:
# Calculate age and working years
application_record['APPLICANT_AGE'] = round(-application_record['DAYS_BIRTH'] / 365)
application_record['YEARS_WORKING'] = round(-application_record['DAYS_EMPLOYED'] / 365)

In [5]:
# Remove applcations < 21 years old
application_record = application_record[application_record['APPLICANT_AGE'] >= 21]  

# Remove rows with null columns
application_record = application_record.dropna()

In [6]:
# Import credit record dataset
credit_record = pd.read_csv("data/credit_record.csv")
credit_record.shape

(1048575, 3)

In [7]:
mapz = {
    'C': 'Good Debt', # Loan paid on time
    'X': 'Good Debt', # No loan for the month
    '0': 'Good Debt', # Loan 1-29 days overdue
    '1': 'Bad Debt', # Loan 30-59 days overdue
    '2': 'Bad Debt', # Loan 60-89 days overdue
    '3': 'Bad Debt', # Loan 90-119 days overdue
    '4': 'Bad Debt', # Loan 120-149 days overdue
    '5': 'Bad Debt', # Loan > 150 days overdue
}
credit_record['STATUS_2'] = credit_record['STATUS'].map(mapz)

credit_record['TOTAL_GOOD_DEBT'] = np.where(credit_record['STATUS_2'].eq('Good Debt', 'Bad Debt'), 1, 0)
credit_record['TOTAL_BAD_DEBT'] = np.where(credit_record['STATUS_2'].eq('Good Debt', 'Bad Debt'), 0, 1)

df2 = credit_record.groupby('ID').sum()

df2['STATUS'] = np.where(df2['TOTAL_GOOD_DEBT'] >= df2['TOTAL_BAD_DEBT'], 1, 0)
df2 = df2.drop(columns=['MONTHS_BALANCE'])

df2['GOOD_PERCENT'] = np.where(df2['TOTAL_BAD_DEBT']!= 0, df2['TOTAL_GOOD_DEBT']/(df2['TOTAL_GOOD_DEBT']+df2['TOTAL_BAD_DEBT']), 1)


# Remove empty debt
df2 = df2.dropna()
df2.head()

,TOTAL_GOOD_DEBT,TOTAL_BAD_DEBT,STATUS,GOOD_PERCENT
ID,,,,
5001711,4,0,1,1.0
5001712,19,0,1,1.0
5001713,22,0,1,1.0
5001714,15,0,1,1.0
5001715,60,0,1,1.0


In [8]:
new_df = pd.merge(application_record, df2, on="ID")
new_df = new_df.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED'])
new_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,APPLICANT_AGE,YEARS_WORKING,TOTAL_GOOD_DEBT,TOTAL_BAD_DEBT,STATUS,GOOD_PERCENT
0,5008806,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,0,0,Security staff,2.0,59.0,3.0,30,0,1,1.0
1,5008808,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1.0,52.0,8.0,5,0,1,1.0
2,5008809,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1.0,52.0,8.0,5,0,1,1.0
3,5008810,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1.0,52.0,8.0,27,0,1,1.0
4,5008811,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1.0,52.0,8.0,39,0,1,1.0


In [9]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25134 entries, 0 to 25133
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   25134 non-null  int64  
 1   CODE_GENDER          25134 non-null  object 
 2   FLAG_OWN_CAR         25134 non-null  int32  
 3   FLAG_OWN_REALTY      25134 non-null  int32  
 4   CNT_CHILDREN         25134 non-null  int64  
 5   AMT_INCOME_TOTAL     25134 non-null  float64
 6   NAME_INCOME_TYPE     25134 non-null  object 
 7   NAME_EDUCATION_TYPE  25134 non-null  object 
 8   NAME_FAMILY_STATUS   25134 non-null  object 
 9   NAME_HOUSING_TYPE    25134 non-null  object 
 10  FLAG_MOBIL           25134 non-null  int64  
 11  FLAG_WORK_PHONE      25134 non-null  int64  
 12  FLAG_PHONE           25134 non-null  int64  
 13  FLAG_EMAIL           25134 non-null  int64  
 14  OCCUPATION_TYPE      25134 non-null  object 
 15  CNT_FAM_MEMBERS      25134 non-null 

In [10]:
#imbalance
countY, countX = new_df.STATUS.value_counts()
print("Ratio of classes is Y : N = ", countY, ":", countX)

Ratio of classes is Y : N =  25048 : 86


In [11]:
#convert to int
new_df = new_df.round(0).astype({"AMT_INCOME_TOTAL":"int","CNT_FAM_MEMBERS":"int","APPLICANT_AGE":"int","YEARS_WORKING":"int","GOOD_PERCENT":"int"})
new_df.info()
new_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25134 entries, 0 to 25133
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   25134 non-null  int64 
 1   CODE_GENDER          25134 non-null  object
 2   FLAG_OWN_CAR         25134 non-null  int32 
 3   FLAG_OWN_REALTY      25134 non-null  int32 
 4   CNT_CHILDREN         25134 non-null  int64 
 5   AMT_INCOME_TOTAL     25134 non-null  int32 
 6   NAME_INCOME_TYPE     25134 non-null  object
 7   NAME_EDUCATION_TYPE  25134 non-null  object
 8   NAME_FAMILY_STATUS   25134 non-null  object
 9   NAME_HOUSING_TYPE    25134 non-null  object
 10  FLAG_MOBIL           25134 non-null  int64 
 11  FLAG_WORK_PHONE      25134 non-null  int64 
 12  FLAG_PHONE           25134 non-null  int64 
 13  FLAG_EMAIL           25134 non-null  int64 
 14  OCCUPATION_TYPE      25134 non-null  object
 15  CNT_FAM_MEMBERS      25134 non-null  int32 
 16  APPL

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,APPLICANT_AGE,YEARS_WORKING,TOTAL_GOOD_DEBT,TOTAL_BAD_DEBT,STATUS,GOOD_PERCENT
0,5008806,M,1,1,0,112500,Working,Secondary / secondary special,Married,House / apartment,...,0,0,Security staff,2,59,3,30,0,1,1
1,5008808,F,0,1,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1,52,8,5,0,1,1
2,5008809,F,0,1,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1,52,8,5,0,1,1
3,5008810,F,0,1,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1,52,8,27,0,1,1
4,5008811,F,0,1,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,1,1,Sales staff,1,52,8,39,0,1,1


In [12]:
new_df.to_csv('data/processed_data.csv', index=False)